# DriftDesk Support KPI Dashboard

This notebook connects to the MySQL support database and renders KPI summaries.

In [ ]:
import os
import pandas as pd
import mysql.connector
import matplotlib.pyplot as plt

plt.style.use('seaborn-v0_8')

conn = mysql.connector.connect(
    host=os.getenv('MYSQL_HOST', 'localhost'),
    port=int(os.getenv('MYSQL_PORT', '3306')),
    user=os.getenv('MYSQL_USER', 'root'),
    password=os.getenv('MYSQL_PASSWORD', ''),
    database=os.getenv('MYSQL_DATABASE', 'driftdesk_support'),
)

def read_sql(query):
    return pd.read_sql(query, conn)


In [ ]:
kpis = read_sql("""
SELECT
  COUNT(*) AS total_tickets,
  SUM(status = 'open') AS open_tickets,
  SUM(status = 'pending') AS pending_tickets,
  SUM(status = 'closed') AS closed_tickets
FROM support_tickets
""")
kpis

In [ ]:
resolution = read_sql("""
SELECT
  AVG(TIMESTAMPDIFF(HOUR, created_at, closed_at)) AS avg_resolution_hours
FROM support_tickets
WHERE closed_at IS NOT NULL
""")
resolution

In [ ]:
by_priority = read_sql("""
SELECT priority, COUNT(*) AS tickets
FROM support_tickets
GROUP BY priority
ORDER BY tickets DESC
""")
by_priority.plot(kind='bar', x='priority', y='tickets', legend=False, title='Tickets by Priority')
plt.show()

In [ ]:
by_channel = read_sql("""
SELECT channel, COUNT(*) AS tickets
FROM support_tickets
GROUP BY channel
ORDER BY tickets DESC
""")
by_channel.plot(kind='pie', y='tickets', labels=by_channel['channel'], autopct='%1.0f%%', ylabel='')
plt.title('Tickets by Channel')
plt.show()

In [ ]:
daily = read_sql("""
SELECT DATE(created_at) AS day, COUNT(*) AS tickets
FROM support_tickets
GROUP BY DATE(created_at)
ORDER BY day
""")
daily.plot(kind='line', x='day', y='tickets', marker='o', title='Daily New Tickets')
plt.xticks(rotation=45)
plt.show()